From previous notebook to import and scrub the data from wikipedia

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#html_data=requests.get(url).text
#soup = BeautifulSoup(html_data.text, 'html.parser')
data = pd.read_html(url)

df_temp = data[0]

In [2]:
def str_breaker(str_in):
    pCode = str_in[0:3]
    if "Not assigned" in str_in:
        borough = "Not assigned"
        neighborhood = ''
    else:
        indy = str_in.index('(')
        borough = str_in[3:indy]
        #neighborhood = str_in[indy+1:-1].split(' / ')
        neighborhood = str_in[indy+1:].replace(' / ',', ')
        neighborhood = neighborhood.replace('(','')
        neighborhood = neighborhood.replace(')',' ')
    return pCode, borough, neighborhood

In [3]:
df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])
for i in range(df_temp.shape[0]):
    for j in range(df_temp.shape[1]):
        pC, bo, nh = str_breaker(df_temp.loc[i,j])
        if bo != "Not assigned":
            df = df.append({'PostalCode':pC, 'Borough':bo, 'Neighborhood':nh}, ignore_index = True)

In [4]:
df2 = pd.read_csv(r"C:\Users\M61299\Documents\Data Science\Geospatial_Coordinates.csv")

In [5]:
df_final = df.join(df2.set_index('Postal Code'),on='PostalCode')

# New Code Begins Here
Import additional libraries (mostly copied from Manhattan Segmenting Lab)

In [6]:
import numpy as np
import json

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

Setup for Foursquare API (Credentials removed for publication)

In [19]:
CLIENT_ID = 'CXK04REOP30TX1AJDHAWETG2NJVWV4FZULDNRZYV0W514MZ0' # your Foursquare ID
CLIENT_SECRET = 'JKP0CWYL5DELPV1PKRO0VZ2E3G5SV5UTJCQXD44XX5G5DUOO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
ACCESS_TOKEN = 'ZAQTWMXJD3MUDMBIQ1OUIWDMELCQMLDJG4GTHHORHYKKQIE2'

Following function was copied from lab as well (updated "Neighborhood" to "PostalCode" where need)

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&oauth_token={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            ACCESS_TOKEN,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Create a data frame of boroughs that contain "Toronto" in their name

In [9]:
toronto_data = df_final[df_final['Borough'].astype(str).str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Following code gets the venues in a given post code

In [18]:
toronto_venues = getNearbyVenues(names=toronto_data['PostalCode'], latitudes=toronto_data['Latitude'], longitudes=toronto_data['Longitude'])

M5A


SSLError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?&client_id=CXK04REOP30TX1AJDHAWETG2NJVWV4FZULDNRZYV0W514MZ0&client_secret=JKP0CWYL5DELPV1PKRO0VZ2E3G5SV5UTJCQXD44XX5G5DUOO&v=20180605&ll=43.6542599,-79.3606359&oauth_token=ZAQTWMXJD3MUDMBIQ1OUIWDMELCQMLDJG4GTHHORHYKKQIE2&radius=500&limit=5 (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))